In [0]:
#Task1: Download the data
import urllib
urllib.request.urlretrieve("https://s3.amazonaws.com/ibotta-data-engineer-test/311_service_requests.csv.zip","/tmp/311_service_requests.csv.zip")
urllib.request.urlretrieve("https://s3.amazonaws.com/ibotta-data-engineer-test/traffic_accidents.csv.zip","/tmp/traffic_accidents.csv.zip")

Out[1]: ('/tmp/traffic_accidents.csv.zip', <http.client.HTTPMessage at 0x7f1b6ea6b650>)

In [0]:
%sh

unzip /tmp/311_service_requests.csv.zip 
unzip /tmp/traffic_accidents.csv.zip

Archive: /tmp/311_service_requests.csv.zip
 inflating: 311_service_requests.csv 
 creating: __MACOSX/
 inflating: __MACOSX/._311_service_requests.csv 
Archive: /tmp/traffic_accidents.csv.zip
 inflating: traffic_accidents.csv 
 inflating: __MACOSX/._traffic_accidents.csv

In [0]:
%sh 
pwd
 ls 


/databricks/driver
311_service_requests.csv
__MACOSX
conf
derby.log
eventlogs
logs
traffic_accidents.csv

In [0]:
dbutils.fs.mv("file:/databricks/driver/311_service_requests.csv","dbfs:/tmp/311_service_requests.csv")
dbutils.fs.mv("file:/databricks/driver/traffic_accidents.csv","dbfs:/tmp/traffic_accidents.csv")

Out[4]: True

In [0]:
#create dataframes
service_requests_df = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/tmp/311_service_requests.csv")
traffic_accidents_df = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/tmp/traffic_accidents.csv")

In [0]:
#Task2: Create a view or table
service_requests_df.createOrReplaceTempView("service_requests")
traffic_accidents_df.createOrReplaceTempView("traffic_accidents")

In [0]:
%sql
--Task3 Traffic Accidents are minimum around the month of Feb/March and maximum around the month of october
select CONCAT(YEAR(REPORTED_DATE),'-',LPAD(MONTH(REPORTED_DATE),2,0)) AS YR_MNTH, COUNT(*) AS CNT 
FROM traffic_accidents 
WHERE YEAR(REPORTED_DATE) IN (2013,2014,2015) 
GROUP BY CONCAT(YEAR(REPORTED_DATE),'-',LPAD(MONTH(REPORTED_DATE),2,0)) 
ORDER BY YR_MNTH ASC;

YR_MNTH,CNT
2013-01,1583
2013-02,1344
2013-03,1440
2013-04,1482
2013-05,1457
2013-06,1470
2013-07,1655
2013-08,1823
2013-09,1810
2013-10,1837


In [0]:
%sql

SELECT DISTINCT OFFENSE_TYPE_ID, OFFENSE_CATEGORY_ID 
FROM traffic_accidents;

OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID
traffic-accident,traffic-accident
traf-vehicular-homicide,murder
traffic-accident-hit-and-run,traffic-accident
traffic-accident-dui-duid,traffic-accident
traf-vehicular-assault,all-other-crimes


In [0]:
%sql
-- Seasonal trend on different type of traffic accidents showing minimum traffic accidents in Feb and maximum accidents in october 
SELECT OFFENSE_CATEGORY_ID, OFFENSE_TYPE_ID, CONCAT(YEAR(REPORTED_DATE),'-',LPAD(MONTH(REPORTED_DATE),2,0)) AS YR_MNTH, COUNT(*) AS CNT 
FROM traffic_accidents WHERE MONTH(REPORTED_DATE) IN (2,10) AND YEAR(REPORTED_DATE) IN (2013,2014,2015) AND OFFENSE_CATEGORY_ID = "traffic-accident" 
GROUP BY CONCAT(YEAR(REPORTED_DATE),'-',LPAD(MONTH(REPORTED_DATE),2,0)), OFFENSE_CATEGORY_ID, OFFENSE_TYPE_ID 
ORDER BY OFFENSE_TYPE_ID ASC, YR_MNTH ASC;

OFFENSE_CATEGORY_ID,OFFENSE_TYPE_ID,YR_MNTH,CNT
traffic-accident,traffic-accident,2013-02,962
traffic-accident,traffic-accident,2013-10,1284
traffic-accident,traffic-accident,2014-02,1117
traffic-accident,traffic-accident,2014-10,1475
traffic-accident,traffic-accident,2015-02,1314
traffic-accident,traffic-accident,2015-10,1549
traffic-accident,traffic-accident-dui-duid,2013-02,49
traffic-accident,traffic-accident-dui-duid,2013-10,67
traffic-accident,traffic-accident-dui-duid,2014-02,66
traffic-accident,traffic-accident-dui-duid,2014-10,76


In [0]:
%sql
-- Maximum service requests are created in summer and minimum in winter
select CONCAT(YEAR(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "mm/dd/yyyy"),"yyyy-mm-dd")),'-',LPAD(MONTH(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "mm/dd/yyyy"),"yyyy-mm-dd")),2,0)) AS YR_MNTH, COUNT(*) AS CNT 
FROM service_requests 
GROUP BY CONCAT(YEAR(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "mm/dd/yyyy"),"yyyy-mm-dd")),'-',LPAD(MONTH(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "mm/dd/yyyy"),"yyyy-mm-dd")),2,0)) 
ORDER BY YR_MNTH ASC;

YR_MNTH,CNT
2015-10,24616
2015-11,35942
2015-12,35819
2016-01,34597
2016-02,37119
2016-03,39565
2016-04,39687
2016-05,39555
2016-06,44740
2016-07,38824


In [0]:
%sql
--Stapleton and Baker neighbourhood areas are high prone to traffic accidents.
SELECT OFFENSE_TYPE_ID, NEIGHBORHOOD_ID, COUNT(*) AS CNT 
FROM traffic_accidents 
GROUP BY NEIGHBORHOOD_ID , OFFENSE_TYPE_ID 
ORDER BY CNT DESC 
LIMIT 10;

OFFENSE_TYPE_ID,NEIGHBORHOOD_ID,CNT
traffic-accident,stapleton,5129
traffic-accident,baker,4738
traffic-accident,hampden-south,3023
traffic-accident,capitol-hill,2697
traffic-accident,lincoln-park,2675
traffic-accident,five-points,2570
traffic-accident,globeville,2108
traffic-accident,highland,2051
traffic-accident,montbello,1980
traffic-accident,cbd,1800


In [0]:
%sql
SELECT Type, `Incident Zip Code`, COUNT(*) AS CNT FROM service_requests  GROUP BY Type, `Incident Zip Code` HAVING CNT > 1 ORDER BY CNT DESC LIMIT 20;

Type,Incident Zip Code,CNT
Inquiry,null,302095
Request,null,31038
null,null,10877
Complaint,null,7856
Request,80219,7171
Request,80211,5329
Request,80204,5133
Request,80205,4823
Request,80210,4652
Request,80220,4169


In [0]:
%sql
SELECT `Incident Address 1` ,COUNT(*) AS CNT FROM service_requests GROUP BY `Incident Address 1` HAVING CNT >1 ORDER BY CNT DESC LIMIT 100;

Incident Address 1,CNT
null,341973
201 W Colfax Ave,501
N/A,372
n/a,331
na,175
NA,138
1234 Nowhere Lane,121
1111,117
9770 E Alameda Ave,103
1700 N Sheridan Blvd,89


In [0]:
%sql
-- Average no.of days for case resolution based on agency
SELECT Agency, AVG(DATEDIFF(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Closed Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"),FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"))) as avg_number_of_days
FROM service_requests
GROUP BY Agency
ORDER BY avg_number_of_days DESC;

Agency,avg_number_of_days
Public Works,2.241087414727013
Parks & Recreation,2.0377219840783836
Mayor's Office,1.4804845222072678
Community Planning & Development,1.3762165450121655
City Council,1.0152477763659467
Tech Services,1.0
Excise & License,0.9371046228710462
Environmental Health,0.8535993762813664
311,0.7938142548596112
Auditor's Office,0.3333333333333333


In [0]:
%sql
--Average case resolution time based on type of case in descending order od resolution time
SELECT `Case Summary`, AVG(DATEDIFF(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Closed Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"),FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"))) as avg_number_of_days
FROM service_requests
GROUP BY `Case Summary`
ORDER BY avg_number_of_days DESC
LIMIT 20;

Case Summary,avg_number_of_days
broken playground equipment at 17th & Utica,54.0
Portable toilet on north side of RUSSEL PARK is turned over and not usable. Location is on 37th mid block,54.0
Caller needs to request that the port-o-pottys be put back in Curtis Park,54.0
City Council Escalation-Gnrl,54.0
trash under trees and around fountain area of Benedict fountain park,54.0
"Graffiti at Berkeley Recreation center, pump house, restroom",53.0
Disability clr,52.0
Wanted to know if the lights in Civic Center Park would be on for an event he is having. He didn't know if they were maintained by Xcel or Parks. The event is tomorrow evening at dusk (6pm) and would like to have those lights on. Please give him a call.,51.0
Park light out at Manley Park (overhead light are not working in Manley park - pole # 124687682),50.0
Flashing School light.,49.0


In [0]:
%sql
--Average case resolution time based on type of case in ascending order of resolution time
SELECT `Case Summary`, AVG(DATEDIFF(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Closed Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"),FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"))) as avg_number_of_days
FROM service_requests
WHERE `Case Closed Date` IS NOT NULL
GROUP BY `Case Summary`
ORDER BY avg_number_of_days ASC
LIMIT 20;

Case Summary,avg_number_of_days
automated cart conversion inquiry,0.0
Medical examiner,0.0
construction on street,0.0
Delivered: Open Records Response,0.0
handicapped placard renewal question,0.0
Jail Visit,0.0
employee inquiry,0.0
titling,0.0
Parcel # 0517204042000,0.0
renew her tags,0.0


In [0]:
%sql
--Average case resolution time based on case source. 
SELECT `Case Source`, AVG(DATEDIFF(FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Closed Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"),FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd"))) as avg_number_of_days
FROM service_requests
GROUP BY `Case Source`
ORDER BY avg_number_of_days DESC
LIMIT 20;

Case Source,avg_number_of_days
Email - Mayor's Office,13.75
Fax,6.0
Other,5.412844036697248
City Council District 5,3.75
City Council District 3,3.75
PocketGov,3.04058921315109
null,2.857142857142857
City Council District 8,2.5
City Council District 1,2.5
USPS Mail,2.2857142857142856


In [0]:
%sql
--Corelation between the two data sets based on incident addresses
DROP TABLE IF EXISTS correlated_table;
CREATE TABLE IF NOT EXISTS correlated_table AS
SELECT `Case Summary` AS case_summary
     , `Case Status` AS case_status
     , `Case Source` AS case_source
     , `Case Created Date` AS case_created_date
     , `Case Closed Date` AS case_closed_date
     , `Incident Address 1` AS incident_address_1
     , `Agency`
     , `INCIDENT_ID`
     , `OFFENSE_TYPE_ID`
     , `OFFENSE_CATEGORY_ID`
     , `FIRST_OCCURRENCE_DATE`
     , `REPORTED_DATE`
     , `INCIDENT_ADDRESS`
     , `NEIGHBORHOOD_ID`
FROM(SELECT *
     FROM(SELECT *
                , ROW_NUMBER() OVER(PARTITION BY `Incident Address 1` ORDER BY FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(`Case Created Date`,1,10), "MM/dd/yyyy"),"yyyy-MM-dd") ASC) AS ROWX
         FROM service_requests
         ) s1
     WHERE s1.ROWX = 1
     ) s
INNER JOIN(SELECT *
          FROM(SELECT *
                     , ROW_NUMBER() OVER(PARTITION BY INCIDENT_ADDRESS ORDER BY FROM_UNIXTIME(UNIX_TIMESTAMP(SUBSTR(FIRST_OCCURRENCE_DATE,1,10), "MM/dd/yyyy"),"yyyy-MM-dd") ASC) AS ROWY
              FROM traffic_accidents
              ) t1
          WHERE t1.ROWY = 1
          ) t
ON UPPER(TRIM(s.`Incident Address 1`)) = UPPER(TRIM(t.`INCIDENT_ADDRESS`));

In [0]:
%sql
SELECT UPPER(case_summary), OFFENSE_TYPE_ID, COUNT(*) AS CNT, CONCAT_WS('-', collect_list(case_created_date)), CONCAT_WS('-', collect_list(REPORTED_DATE))
FROM correlated_table
GROUP BY UPPER(case_summary), OFFENSE_TYPE_ID
ORDER BY CNT DESC, UPPER(case_summary) ASC, OFFENSE_TYPE_ID ASC

upper(case_summary),OFFENSE_TYPE_ID,CNT,"concat_ws(-, collect_list(case_created_date))","concat_ws(-, collect_list(REPORTED_DATE))"
GRAFFITI,traffic-accident-hit-and-run,116,09/30/2016-03/04/2016-12/02/2015-04/06/2016-01/17/2016-04/25/2016-06/13/2016-12/02/2015-04/04/2016-07/28/2016-01/29/2016-06/15/2016-11/24/2015-04/27/2016-10/11/2016-11/21/2015-06/23/2016-02/02/2016-09/24/2016-02/11/2016-05/09/2016-02/10/2016-12/29/2015-03/15/2016-04/01/2016-02/22/2016-05/19/2016-12/28/2015-04/25/2016-05/21/2016-06/03/2016-11/16/2015-05/22/2016-12/29/2015-08/20/2016-02/16/2016-04/13/2016-09/02/2016-12/05/2015-02/17/2016-11/19/2015-08/26/2016-01/20/2016-02/03/2016-03/09/2016-12/03/2015-01/15/2016-12/18/2015-12/09/2015-04/01/2016-03/11/2016-12/02/2015-01/16/2016-10/03/2016-04/23/2016-07/19/2016-11/30/2015-12/07/2015-06/02/2016-02/10/2016-08/26/2016-02/25/2016-07/12/2016-08/09/2016-12/05/2015-02/16/2016-04/19/2016-08/03/2016-09/29/2016-12/14/2015-12/02/2015-11/16/2015-04/06/2016-06/01/2016-01/19/2016-07/01/2016-01/12/2016-02/25/2016-10/05/2016-04/21/2016-01/14/2016-11/22/2015-03/10/2016-10/05/2016-11/22/2015-07/22/2016-08/11/2016-02/09/2016-03/10/2016-01/22/2016-12/31/2015-06/06/2016-06/22/2016-11/17/2015-02/16/2016-12/18/2015-09/23/2016-11/19/2015-01/07/2016-05/07/2016-05/01/2016-03/31/2016-03/14/2016-09/21/2016-01/04/2016-09/07/2016-12/03/2015-01/19/2016-05/29/2016-11/19/2015-01/21/2016-05/06/2016-10/10/2016-02/16/2016-11/23/2015-06/13/2016,2015-08-27 10:18:59-2011-01-04 14:25:00-2015-02-27 18:52:00-2015-11-16 18:11:00-2016-03-11 10:09:59-2013-05-21 22:02:00-2016-07-21 09:14:59-2011-04-07 08:38:00-2013-01-26 15:30:59-2012-04-13 15:01:59-2012-10-12 12:20:59-2013-11-21 14:48:59-2012-12-29 15:43:59-2014-10-13 07:44:00-2012-04-20 19:57:59-2011-10-01 16:38:59-2011-04-21 13:18:00-2011-09-07 17:13:00-2011-07-17 20:39:59-2015-07-14 11:13:59-2013-12-28 00:34:59-2013-05-30 15:27:00-2011-10-13 07:24:00-2011-10-16 19:05:59-2011-12-03 12:02:59-2014-01-01 01:05:00-2016-08-23 15:58:00-2014-11-20 12:33:59-2015-07-08 15:30:00-2011-04-08 20:16:00-2015-08-31 00:30:59-2014-01-30 16:27:00-2012-12-10 02:39:59-2016-09-06 16:54:00-2013-05-12 22:34:59-2011-09-17 03:50:59-2015-04-11 17:32:00-2012-08-30 20:29:59-2011-07-08 15:27:59-2013-07-24 17:55:59-2014-07-08 15:31:59-2012-04-21 08:13:59-2016-03-02 10:57:59-2012-08-06 11:01:59-2014-11-01 06:53:59-2016-07-19 08:58:59-2014-08-02 19:53:00-2013-05-09 14:39:59-2015-07-08 15:27:59-2012-10-20 18:37:59-2016-06-30 13:41:59-2012-02-06 14:19:59-2014-06-15 16:51:59-2012-02-03 22:48:59-2012-06-17 16:25:59-2013-11-05 09:36:59-2011-07-02 18:11:00-2014-02-23 12:18:59-2011-07-27 12:04:59-2012-04-07 20:13:00-2011-04-19 11:01:59-2015-07-08 09:13:00-2012-08-05 21:11:59-2015-02-02 16:50:59-2016-01-15 17:23:00-2011-02-03 11:07:59-2012-01-12 17:36:59-2011-06-20 19:56:00-2011-03-16 17:12:00-2015-11-11 11:45:00-2015-02-01 05:13:00-2015-11-21 00:20:00-2011-10-27 15:11:59-2011-05-08 22:03:59-2014-08-08 14:41:00-2015-02-22 07:06:59-2012-02-04 15:36:00-2015-08-12 10:11:00-2011-07-20 19:37:00-2012-06-26 11:16:00-2011-06-28 09:29:00-2012-01-15 13:30:00-2016-05-14 19:12:00-2011-09-09 18:49:59-2012-02-29 11:52:59-2016-07-23 19:00:59-2014-06-08 03:49:00-2013-09-12 08:51:59-2012-04-09 13:09:59-2011-08-17 02:51:00-2013-04-29 09:31:59-2012-06-07 19:50:59-2011-01-11 16:34:59-2016-07-30 12:33:59-2014-09-27 18:08:59-2015-06-07 22:05:00-2012-12-07 04:05:59-2012-06-06 20:46:59-2013-01-17 15:47:59-2012-06-23 16:12:59-2015-03-21 01:47:00-2014-01-20 15:39:00-2016-10-18 17:45:59-2016-02-28 14:33:59-2012-10-16 14:19:59-2015-08-23 17:35:00-2012-11-02 14:42:00-2014-04-28 12:11:59-2016-02-27 16:02:00-2016-01-21 14:07:59-2014-08-02 19:53:00-2016-07-21 09:17:59-2011-02-25 10:51:59-2015-04-04 12:59:00-2011-09-01 15:53:00-2013-05-23 14:12:00
GRAFFITI,traffic-accident,39,05/23/2016-01/20/2016-11/22/2015-04/15/2016-01/19/2016-05/06/2016-01/05/2016-12/15/2015-06/13/2016-09/05/2016-04/06/2016-01/13/2016-03/31/2016-12/10/2015-12/26/2015-11/25/2015-12/04/2015-11/20/2015-04/